In [218]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE,RandomOverSampler
from xgboost import XGBClassifier


In [219]:
train_data = pd.read_csv('dataset2-updated.csv')

In [220]:
train_data.drop(['file_id', 'user_id','subforum_id','num_contexts'], axis = 1,inplace = True)

In [221]:
def preprocess(tweet):
   
   # Removing handles and hashtags
   tweet = re.sub('@[^\s]+','',str(tweet))
   tweet = re.sub(r'#', '', str(tweet))
   # Removing URLS
   tweet= re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',str(tweet))
   #removing all punchuation and special character
   tweet = re.sub('[^a-zA-Z]',' ',str(tweet))
   #removing extra white space
   tweet = re.sub('[\s]+', ' ', str(tweet))
   tweet = re.sub('[\n]+', ' ', str(tweet))
   tweet=tweet.lower()
   return tweet
train_data['tweet']= train_data['tweet'].apply(lambda x:preprocess(x))
train_data.head()

,label,tweet
0,noHate,as of march th the booklet had been downloaded...
1,noHate,thank you in advance download the youtube desc...
2,noHate,in order to help increase the booklets downloa...
3,noHate,simply copy and paste the following text into...
4,noHate,click below for a free download of a colorfull...


In [222]:
train_data.shape

(11089, 2)

In [223]:
#Performing Tokenization
def token(tweet):
  tweet = word_tokenize(tweet)
  return tweet
train_data['tweet']=train_data['tweet'].apply(lambda x:token(x))

In [224]:
#Performing Text Normalization
lem = WordNetLemmatizer()
def normalize(tweet):
  tweet = [lem.lemmatize(word) for word in tweet if not word in stopwords.words('english')]
  #print(tweet)
  tweet = ' '.join(tweet)
  return tweet
train_data['tweet']= train_data['tweet'].apply(lambda x :normalize(x))
train_data.dropna(inplace=True)

In [225]:
#TF-IDF
x_tf=train_data['tweet']
y_tf=train_data['label']

In [226]:
x_train,x_test,y_train,y_test= train_test_split(x_tf,y_tf,test_size=.2,random_state=1)

In [227]:
# Transforming Text Using TFIDF
tfidf = TfidfVectorizer(ngram_range=(1,3),max_features=10000)
tfidf.fit(x_train)

TfidfVectorizer(max_features=10000, ngram_range=(1, 3))

In [228]:
x_train = tfidf.transform(x_train)
x_test = tfidf.transform(x_test)

In [229]:
pd.set_option('display.max_columns', None)
df_tf=pd.DataFrame(x_train.toarray(),columns=tfidf.get_feature_names())
df_tf.head()

,ability,abit,able,able get,able see,able survive,aboard,abortion,abroad,absolute,absolutely,abuse,abusive,accept,acceptable,accepted,accepting,access,access airwave,access airwave per,access tv program,accident,accomplish,according,account,accurate,accusation,accused,achieve,achievement,acknowledge,acquaintance,across,across border,across country,across nation,across site,act,act aria,act like,acted,acting,acting like,action,active,actively,activism,activist,activity,actor,actress,actual,actually,actually read,actualy,ad,adam,add,added,addict,adding,address,adj,adjective,admin,administration,admission,admit,admits,admits crime,adopt,adopted,adoption,adult,advance,advance scout,advanced,advanced scout,advanced scout forum,advancement,advantage,adventure,adversary,advert,advertisement,advice,advise,advocate,affirmative,affirmative action,afford,affordable,afraid,africa,africa black,african,afternoon,age,aged,aged son,agency,agenda,agent,ago,ago got,ago small,ago someone,ago still,agree,agreed,agreement,ah,ahead,ai,aid,aim,aint,air,air force,air walt,air walt disney,airport,airwave per thread,ak,aka,al,alabama,alan,alarm,albanian,alberta,album,alcohol,alex,alex jones,algebra college,alien,alike,alive,allegedly,allegiance,allen,alliance,allied,allow,allowed,allowing,ally,almost,almost always,almost every,almost everyone,alone,along,along line,alot,alot time,alpine,already,already know,alright,also,also get,also looking,also lot,also make,also want,alt,alt right,alt right poster,altar,alternative,although,aluminum,always,always always,always get,always laugh,always nice,always nice meet,always nice see,always one,always said,always thought,amazing,amazon,amazon co,amazon co uk,amazon com,amazon com europe,amazon com scotch,amazonaws,amazonaws com,amazonaws com ksr,amendment,america,america murder,american,american flag,american internment,american school,ammo,among,amongst,amount,amount non,amount non white,amount time,ample,ample supply,amy,analogy,analysis,ancestor,ancestry,ancient,ancient egyptian,andrew,andy,angeles,anger,anglo,angry,animal,anime,announced,annoying,another,another article,another day,another example,another great,another hand,another thread,answer,answered,answering,ant,anthony,anthropologist,anti,anti fascist,anti gay,anti racism,anti racist,anti semitism,anti white,antwoord,anybody,anybody else,anymore,anyone,anyone belief,anyone else,anyone ever,anyone going,anyone interested,anyone interested pm,anyone know,anyone life,anyone really,anyone tell,anyone want,anyone white,anything,anything else,anything go,anything like,anything white,anytime,anyway,anyways,anywhere,anywhere world,aol,ap,apart,apartment,apartment complex,ape,ape gorilla,apologize,apparently,appear,appearance,appears,apple,application,apply,appreciate,appropriate,approved,approximately,approximately two,approximately two thousand,april,apt,ar,arab,arabic,aragorn,archive,area,arent,argent,argument,arkansas,arm,arm leg,armed,armenian,armor,army,around,around age,around bit,around black,around country,around friend,around world,arrest,arrested,arrested year,arrested year old,arrived,arrogant,arse,arseniy,arseniy yatsenyuk,art,article,artist,artistic,aryan,aryan body,aryan body mind,aryan woman,as,asap,ashamed,ashore,asia,asian,asian art,asian black,asian white,aside,ask,asked,asking,aspect,ass,assassin,assassin part,assassin part youtube,assault,assaulted,asset,asset jpg,associate,associated,assuming,assumption,astorga,aswell,asylum,ate,athlete,athletic,athletic white,ation,atlanta,atleast,attached,attack,attack western,attack western civilization,attack white,attacked,attacking,attempt,attempted,attend,attended,attending,attention,attitude,attractive,audience,audio,aufidena,aug,august,aunt,aussie,austin,austin texas,australia,australian,author,authority,authority said,auto,available,ave,average,average white,avoid,award,aware,away,awesome,awful,awhile,axis,az,baby,bach,bachelor,back,back africa,back always,back day,back home,back school,back txt

## Random Forest

In [14]:
# Used GridSearchCv And Trained With the best parameters 
rtree_tf1=RandomForestClassifier(max_depth=2,n_estimators=50,random_state=4,min_samples_split=5,max_leaf_nodes=30,min_samples_leaf=10,max_features=80)
rtree_tf1.fit(x_train,y_train)

RandomForestClassifier(max_depth=2, max_features=80, max_leaf_nodes=30,
                       min_samples_leaf=10, min_samples_split=5,
                       n_estimators=50, random_state=4)

In [15]:
print(rtree_tf1.score(x_train,y_train))
print(rtree_tf1.score(x_test,y_test))

0.8904294893473115
0.8958521190261497


In [16]:
rtreepred=rtree_tf1.predict(x_test)

In [17]:
pd.DataFrame(confusion_matrix(rtreepred,y_test))

,0,1
0,0,0
1,231,1987


In [18]:
print(classification_report(y_test,rtreepred))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       231
      noHate       0.90      1.00      0.95      1987

    accuracy                           0.90      2218
   macro avg       0.45      0.50      0.47      2218
weighted avg       0.80      0.90      0.85      2218



C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## KNN

In [19]:
# Used GridSearchCv And Trained With the best parameters 
knn_tf1=KNeighborsClassifier(n_neighbors=15)
knn_tf1.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=15)

In [20]:
print(knn_tf1.score(x_train,y_train))
print(knn_tf1.score(x_test,y_test))

0.8904294893473115
0.8958521190261497


In [21]:
knn_preds=knn_tf1.predict(x_test)
pd.DataFrame(confusion_matrix(knn_preds,y_test))

,0,1
0,0,0
1,231,1987


In [22]:
print(classification_report(y_test,knn_preds))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       231
      noHate       0.90      1.00      0.95      1987

    accuracy                           0.90      2218
   macro avg       0.45      0.50      0.47      2218
weighted avg       0.80      0.90      0.85      2218



C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM

In [23]:
# Used GridSearchCv And Trained With the best parameters 
svm_tf1=SVC(C=10,gamma=0.05,kernel='rbf')
svm_tf1.fit(x_train,y_train)

SVC(C=10, gamma=0.05)

In [24]:
print(svm_tf1.score(x_train,y_train))
print(svm_tf1.score(x_test,y_test))

0.9351820538834404
0.9035166816952209


In [25]:
svm_pred=svm_tf1.predict(x_test)
pd.DataFrame(confusion_matrix(svm_pred,y_test))

,0,1
0,39,22
1,192,1965


In [26]:
print(classification_report(y_test,svm_pred))

              precision    recall  f1-score   support

        hate       0.64      0.17      0.27       231
      noHate       0.91      0.99      0.95      1987

    accuracy                           0.90      2218
   macro avg       0.78      0.58      0.61      2218
weighted avg       0.88      0.90      0.88      2218



## XG BOOST

In [27]:
# Used GridSearchCv And Trained With the best parameters 
xgb_tf1=XGBClassifier(max_depth=3,n_estimators=50,learning_rate=0.5)
xgb_tf1.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
print(xgb_tf1.score(x_train,y_train))
print(xgb_tf1.score(x_test,y_test))

0.9206402885807689
0.8994589720468891


In [29]:
xgb_pred=xgb_tf1.predict(x_test)
pd.DataFrame(confusion_matrix(xgb_pred,y_test))

,0,1
0,40,32
1,191,1955


In [30]:
print(classification_report(y_test,xgb_pred))

              precision    recall  f1-score   support

        hate       0.56      0.17      0.26       231
      noHate       0.91      0.98      0.95      1987

    accuracy                           0.90      2218
   macro avg       0.73      0.58      0.61      2218
weighted avg       0.87      0.90      0.88      2218



## ADA BOOST

In [31]:
from sklearn.ensemble import AdaBoostClassifier
clf_ada = AdaBoostClassifier(n_estimators=300,learning_rate=0.5,random_state=0)
clf_ada.fit(x_train,y_train)

print(clf_ada.score(x_train,y_train))
print(clf_ada.score(x_test,y_test))

pred_ada = clf_ada.predict(x_test)
pd.DataFrame(confusion_matrix(pred_ada,y_test))

0.9292075301544358
0.8913435527502255


,0,1
0,43,53
1,188,1934


In [32]:
print(classification_report(y_test,pred_ada))

              precision    recall  f1-score   support

        hate       0.45      0.19      0.26       231
      noHate       0.91      0.97      0.94      1987

    accuracy                           0.89      2218
   macro avg       0.68      0.58      0.60      2218
weighted avg       0.86      0.89      0.87      2218



## Gradient Boost

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
clf1_gb = GradientBoostingClassifier(random_state=0,n_estimators=300,learning_rate=0.01,max_leaf_nodes=40,max_depth=10)
clf1_gb.fit(x_train,y_train)

GradientBoostingClassifier(learning_rate=0.01, max_depth=10, max_leaf_nodes=40,
                           n_estimators=300, random_state=0)

In [34]:
print(clf1_gb.score(x_train,y_train))
print(clf1_gb.score(x_test,y_test))


0.9217675572088829
0.8963029756537421


In [35]:
pred_gb = clf1_gb.predict(x_test)
pd.DataFrame(confusion_matrix(pred_gb,y_test))

,0,1
0,23,22
1,208,1965


In [36]:
print(classification_report(y_test,pred_gb))

              precision    recall  f1-score   support

        hate       0.51      0.10      0.17       231
      noHate       0.90      0.99      0.94      1987

    accuracy                           0.90      2218
   macro avg       0.71      0.54      0.56      2218
weighted avg       0.86      0.90      0.86      2218



In [37]:
cm_random_forest=confusion_matrix(y_test,rtreepred)
cm_knn=confusion_matrix(y_test,knn_preds)
cm_svm=confusion_matrix(y_test,svm_pred)
cm_xgboost =confusion_matrix(y_test,xgb_pred)
cm_adaboost=confusion_matrix(pred_ada,y_test)
cm_gradientboost=confusion_matrix(pred_gb,y_test)

In [38]:
print('Without Sampling')
print('------------------------')
print('Random Forest')
print('Training Accuracy :',rtree_tf1.score(x_train,y_train))
print('Testing Accuracy :',rtree_tf1.score(x_test,y_test))
print('-------------------------')
print('KNN')
print('Training Accuracy :',knn_tf1.score(x_train,y_train))
print('Testing Accuracy :',knn_tf1.score(x_test,y_test))
print('-------------------------')
print('SVM')

print('Training Accuracy :',svm_tf1.score(x_train,y_train))
print('Testing Accuracy :',svm_tf1.score(x_test,y_test))

print('-------------------------')
print('XGBoost')
print('Training Accuracy :',xgb_tf1.score(x_train,y_train))
print('Testing Accuracy :',xgb_tf1.score(x_test,y_test))

print('-------------------------')
print('AdaBoost')
print('Training Accuracy :',clf_ada.score(x_train,y_train))
print('Testing Accuracy :',clf_ada.score(x_test,y_test))

print('-------------------------')
print('GradientBoost')
print('Training Accuracy :',clf1_gb.score(x_train,y_train))
print('Testing Accuracy :',clf1_gb.score(x_test,y_test))


Without Sampling
------------------------
Random Forest
Training Accuracy : 0.8904294893473115
Testing Accuracy : 0.8958521190261497
-------------------------
KNN
Training Accuracy : 0.8904294893473115
Testing Accuracy : 0.8958521190261497
-------------------------
SVM
Training Accuracy : 0.9351820538834404
Testing Accuracy : 0.9035166816952209
-------------------------
XGBoost
Training Accuracy : 0.9206402885807689
Testing Accuracy : 0.8994589720468891
-------------------------
AdaBoost
Training Accuracy : 0.9292075301544358
Testing Accuracy : 0.8913435527502255
-------------------------
GradientBoost
Training Accuracy : 0.9217675572088829
Testing Accuracy : 0.8963029756537421


In [39]:
print('Without Sampling')
print('-------------------------')
print('Confusion Maxtrix For Random Forest')
print(cm_random_forest)
print('-----------------------------------')
print('Confusion Maxtrix For KNN')
print(cm_knn)
print('--------------------------------------')
print('Confusion Maxtrix For SVM')
print(cm_svm)
print('----------------------------')
print('Confusion Maxtrix For XGB')
print(cm_xgboost)
print('----------------------------')
print('Confusion Maxtrix For AdaBoost')
print(cm_adaboost)
print('----------------------------')
print('Confusion Maxtrix For GB')
print(cm_gradientboost)

Without Sampling
-------------------------
Confusion Maxtrix For Random Forest
[[   0  231]
 [   0 1987]]
-----------------------------------
Confusion Maxtrix For KNN
[[   0  231]
 [   0 1987]]
--------------------------------------
Confusion Maxtrix For SVM
[[  39  192]
 [  22 1965]]
----------------------------
Confusion Maxtrix For XGB
[[  40  191]
 [  32 1955]]
----------------------------
Confusion Maxtrix For AdaBoost
[[  43   53]
 [ 188 1934]]
----------------------------
Confusion Maxtrix For GB
[[  23   22]
 [ 208 1965]]


## Using TFIDF With Sampling

In [230]:
ros=RandomOverSampler(random_state=1)
x_ros,y_ros=ros.fit_sample(x_train,y_train)
x_train_tf,x_test_tf,y_train_tf,y_test_tf= train_test_split(x_ros,y_ros,test_size=.2,random_state=1)

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


# Random Forest

In [41]:
# Used GridSearchCv And Trained With the best parameters 
rtree_tf2 = RandomForestClassifier(max_depth=12,n_estimators=500,random_state=13,min_samples_split=5,max_leaf_nodes=30,min_samples_leaf=10,max_features=80)
rtree_tf2.fit(x_train_tf,y_train_tf)

RandomForestClassifier(max_depth=12, max_features=80, max_leaf_nodes=30,
                       min_samples_leaf=10, min_samples_split=5,
                       n_estimators=500, random_state=13)

In [42]:
print(rtree_tf2.score(x_train_tf,y_train_tf))
print(rtree_tf2.score(x_test_tf,y_test_tf))

0.8036081658490267
0.785126582278481


In [43]:
ypred=rtree_tf2.predict(x_test_tf)

In [44]:
pd.DataFrame(confusion_matrix(ypred,y_test_tf))

,0,1
0,1091,141
1,538,1390


In [45]:
print(classification_report(y_test_tf,ypred))

              precision    recall  f1-score   support

        hate       0.89      0.67      0.76      1629
      noHate       0.72      0.91      0.80      1531

    accuracy                           0.79      3160
   macro avg       0.80      0.79      0.78      3160
weighted avg       0.81      0.79      0.78      3160



## XG Boost

In [46]:
# Used GridSearchCv And Trained With the best parameters 
xgb_tf2=XGBClassifier(max_depth=11,n_estimators=50,learning_rate=0.01,min_child_weight=1.5,max_leaf_nodes=15,alpha=2)
xgb_tf2.fit(x_train_tf,y_train_tf)

[15:51:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { max_leaf_nodes } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=2, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=11,
              max_leaf_nodes=15, min_child_weight=1.5, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=2, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [47]:
print(xgb_tf2.score(x_train_tf,y_train_tf))
print(xgb_tf2.score(x_test_tf,y_test_tf))

0.7422851717043836
0.7205696202531645


In [48]:
xgbpred = xgb_tf2.predict(x_test_tf)
pd.DataFrame(confusion_matrix(xgbpred,y_test_tf))

,0,1
0,933,187
1,696,1344


In [49]:
print(classification_report(y_test_tf,xgbpred))

              precision    recall  f1-score   support

        hate       0.83      0.57      0.68      1629
      noHate       0.66      0.88      0.75      1531

    accuracy                           0.72      3160
   macro avg       0.75      0.73      0.72      3160
weighted avg       0.75      0.72      0.71      3160



## KNN

In [50]:
# Used GridSearchCv And Trained With the best parameters 
knn_tf2=KNeighborsClassifier(n_neighbors=15,weights='uniform')
knn_tf2.fit(x_train_tf,y_train_tf)

KNeighborsClassifier(n_neighbors=15)

In [51]:
print(knn_tf2.score(x_train_tf,y_train_tf))
print(knn_tf2.score(x_test_tf,y_test_tf))

0.7275676531096692
0.6348101265822785


In [52]:
knnpred=knn_tf2.predict(x_test_tf)
pd.DataFrame(confusion_matrix(knnpred,y_test_tf))

,0,1
0,486,11
1,1143,1520


In [53]:
print(classification_report(y_test_tf,knnpred))

              precision    recall  f1-score   support

        hate       0.98      0.30      0.46      1629
      noHate       0.57      0.99      0.72      1531

    accuracy                           0.63      3160
   macro avg       0.77      0.65      0.59      3160
weighted avg       0.78      0.63      0.59      3160



## SVM

In [231]:
# Used GridSearchCv And Trained With the best parameters 
svm_tf2=SVC(C=10,gamma=1,kernel ='rbf')
svm_tf2.fit(x_train_tf,y_train_tf)

SVC(C=10, gamma=1)

In [232]:
svmpred=svm_tf2.predict(x_test_tf)
print('Training Accuracy For SVM :',svm_tf2.score(x_train_tf,y_train_tf))
print('Testing Accuracy For SVM :',svm_tf2.score(x_test_tf,y_test_tf))

Training Accuracy For SVM : 0.9991296091153663
Testing Accuracy For SVM : 0.9924050632911392


In [233]:
svmpred=svm_tf2.predict(x_test_tf)
pd.DataFrame(confusion_matrix(svmpred,y_test_tf))

,0,1
0,1627,22
1,2,1509


In [234]:
print(classification_report(y_test_tf,svmpred))

              precision    recall  f1-score   support

        hate       0.99      1.00      0.99      1629
      noHate       1.00      0.99      0.99      1531

    accuracy                           0.99      3160
   macro avg       0.99      0.99      0.99      3160
weighted avg       0.99      0.99      0.99      3160



In [152]:
'''
from sklearn.model_selection import GridSearchCV 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}
svm_grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
svm_grid.fit(x_train_tf,y_train_tf)
'''

"\nfrom sklearn.model_selection import GridSearchCV \nparam_grid = {'C': [0.1, 1, 10, 100, 1000],  \n              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], \n              'kernel': ['rbf']}\nsvm_grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) \n  \n# fitting the model for grid search \nsvm_grid.fit(x_train_tf,y_train_tf)\n"

In [154]:
#print(svm_grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
#print(svm_grid.best_estimator_)

In [156]:
#grid_predictions = svm_grid.predict(x_test_tf) 
  
# print classification report 
#print(classification_report(y_test_tf, grid_predictions)) 

## Ada Boost

In [58]:
from sklearn.ensemble import AdaBoostClassifier
clf_ada1 = AdaBoostClassifier(n_estimators=300,learning_rate=0.5,random_state=0)
clf_ada1.fit(x_train_tf,y_train_tf)

print(clf_ada1.score(x_train_tf,y_train_tf))
print(clf_ada1.score(x_test_tf,y_test_tf))

pred_ada1 = clf_ada1.predict(x_test_tf)
pd.DataFrame(confusion_matrix(pred_ada1,y_test_tf))

0.8866118056654534
0.8544303797468354


,0,1
0,1318,149
1,311,1382


In [59]:
print(classification_report(y_test_tf,pred_ada1))

              precision    recall  f1-score   support

        hate       0.90      0.81      0.85      1629
      noHate       0.82      0.90      0.86      1531

    accuracy                           0.85      3160
   macro avg       0.86      0.86      0.85      3160
weighted avg       0.86      0.85      0.85      3160



## Gradient Boost

In [60]:
from sklearn.ensemble import GradientBoostingClassifier
clf1_gb1 = GradientBoostingClassifier(random_state=0,n_estimators=300,learning_rate=0.06,max_leaf_nodes=40,max_depth=10)
clf1_gb1.fit(x_train_tf,y_train_tf)

print(clf1_gb1.score(x_train_tf,y_train_tf))
print(clf1_gb1.score(x_test_tf,y_test_tf))


pred_gb1 = clf1_gb1.predict(x_test_tf)
pd.DataFrame(confusion_matrix(pred_gb1,y_test_tf))

0.9675581579363823
0.9174050632911392


,0,1
0,1527,159
1,102,1372


In [61]:
print(classification_report(y_test_tf,pred_gb1))

              precision    recall  f1-score   support

        hate       0.91      0.94      0.92      1629
      noHate       0.93      0.90      0.91      1531

    accuracy                           0.92      3160
   macro avg       0.92      0.92      0.92      3160
weighted avg       0.92      0.92      0.92      3160



In [62]:
cm_random_forest=confusion_matrix(y_test_tf,ypred)
cm_knn=confusion_matrix(y_test_tf,knnpred)
cm_svm=confusion_matrix(y_test_tf,svmpred)
cm_xgboost=confusion_matrix(y_test_tf,xgbpred)
cm_adaboost=confusion_matrix(pred_ada1,y_test_tf)
cm_gradientboost=confusion_matrix(pred_gb1,y_test_tf)

In [63]:
print('With Sampling')
print('-----------------------------------')
print('Confusion Maxtrix For Random Forest')
print(cm_random_forest)
print('-----------------------------------')
print('Confusion Maxtrix For KNN')
print(cm_knn)
print('--------------------------------------')
print('Confusion Maxtrix For SVM')
print(cm_svm)
print('----------------------------')
print('Confusion Maxtrix For XGB')
print(cm_xgboost)
print('----------------------------')
print('Confusion Maxtrix For AdaBoost')
print(cm_adaboost)
print('----------------------------')
print('Confusion Maxtrix For GB')
print(cm_gradientboost)

With Sampling
-----------------------------------
Confusion Maxtrix For Random Forest
[[1091  538]
 [ 141 1390]]
-----------------------------------
Confusion Maxtrix For KNN
[[ 486 1143]
 [  11 1520]]
--------------------------------------
Confusion Maxtrix For SVM
[[1627    2]
 [  22 1509]]
----------------------------
Confusion Maxtrix For XGB
[[ 933  696]
 [ 187 1344]]
----------------------------
Confusion Maxtrix For AdaBoost
[[1318  149]
 [ 311 1382]]
----------------------------
Confusion Maxtrix For GB
[[1527  159]
 [ 102 1372]]


In [64]:
print('With Sampling')
print('------------------------')
print('Random Forest')
print('Training Accuracy :',rtree_tf2.score(x_train_tf,y_train_tf))
print('Testing Accuracy :',rtree_tf2.score(x_test_tf,y_test_tf))
print('-------------------------')
print('KNN')
print('Training Accuracy :',knn_tf2.score(x_train_tf,y_train_tf))
print('Testing Accuracy :',knn_tf2.score(x_test_tf,y_test_tf))
print('-------------------------')
print('SVM')

print('Training Accuracy :',svm_tf2.score(x_train_tf,y_train_tf))
print('Testing Accuracy :',svm_tf2.score(x_test_tf,y_test_tf))

print('-------------------------')
print('XGBoost')
print('Training Accuracy :',xgb_tf2.score(x_train_tf,y_train_tf))
print('Testing Accuracy :',xgb_tf2.score(x_test_tf,y_test_tf))

print('-------------------------')
print('AdaBoost')
print('Training Accuracy :',clf_ada1.score(x_train_tf,y_train_tf))
print('Testing Accuracy :',clf_ada1.score(x_test_tf,y_test_tf))

print('-------------------------')
print('GradientBoost')
print('Training Accuracy :',clf1_gb1.score(x_train_tf,y_train_tf))
print('Testing Accuracy :',clf1_gb1.score(x_test_tf,y_test_tf))

With Sampling
------------------------
Random Forest
Training Accuracy : 0.8036081658490267
Testing Accuracy : 0.785126582278481
-------------------------
KNN
Training Accuracy : 0.7275676531096692
Testing Accuracy : 0.6348101265822785
-------------------------
SVM
Training Accuracy : 0.9991296091153663
Testing Accuracy : 0.9924050632911392
-------------------------
XGBoost
Training Accuracy : 0.7422851717043836
Testing Accuracy : 0.7205696202531645
-------------------------
AdaBoost
Training Accuracy : 0.8866118056654534
Testing Accuracy : 0.8544303797468354
-------------------------
GradientBoost
Training Accuracy : 0.9675581579363823
Testing Accuracy : 0.9174050632911392


## WORD2VEC Without Sampling

In [65]:
# Making Corpus for Word2Vec
list_of_sent=[] 
for sent in train_data['tweet'].values:
   list_of_sent.append(sent)

In [66]:
import gensim
from gensim.models import Word2Vec
w2v_model=gensim.models.Word2Vec(list_of_sent,workers=3,size=100,min_count=40,window=10,sample=1e-3)

In [67]:
def make_feature_vec(words):
  #for sent in list_of_sent:
  feature_vec=np.zeros((100,),dtype='float32')
  nwords = 0
  index2word_set= set(w2v_model.wv.index2word) 
  for word in words:
    if word in index2word_set:
        nwords +=1
        feature_vec=np.add(feature_vec,w2v_model[word])     
  feature_vec=np.divide(feature_vec,nwords)
  feature_vec = np.around(feature_vec,3)
  return feature_vec

In [68]:
def get_avg_feature_vec(tweets):
    c=0
    tweet_feature_vec = np.zeros((len(tweets),100),dtype='float32')
    for tweet in tweets:
      tweet_feature_vec[c]=make_feature_vec(tweet)
      c=c+1
    return tweet_feature_vec
#tweets=[]
#for tweet in df['tweet']:
  #tweets.append(tweet)
x= get_avg_feature_vec(train_data['tweet'])

C:\Users\pradeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
C:\Users\pradeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [69]:
df2=pd.DataFrame(x)
df2['labels'] = train_data['label']

In [70]:
df2.dropna(inplace=True)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,labels
0,-0.069,-0.067,-0.074,0.056,-0.046,0.126,-0.181,0.034,0.025,0.051,-0.002,-0.004,0.197,0.047,-0.067,-0.053,-0.065,-0.097,0.049,-0.087,-0.018,-0.012,0.134,0.027,-0.052,0.031,-0.012,-0.014,-0.003,-0.001,0.008,-0.178,-0.015,-0.025,-0.062,-0.009,0.057,-0.016,-0.178,0.000,0.015,-0.077,-0.050,-0.052,-0.029,-0.136,0.015,0.022,-0.217,-0.151,-0.152,-0.089,-0.061,-0.111,0.003,0.051,-0.074,0.041,-0.088,0.031,-0.038,0.094,0.035,0.038,0.031,0.021,0.104,0.078,0.135,0.047,0.023,-0.151,-0.039,0.067,-0.104,0.140,-0.044,-0.144,-0.022,0.071,0.109,0.202,-0.071,0.117,0.096,-0.098,-0.033,0.023,0.121,0.025,0.005,0.007,0.136,-0.025,-0.160,0.063,-0.023,0.041,0.077,0.102,noHate
1,-0.045,-0.059,-0.062,0.058,-0.030,0.100,-0.143,0.012,0.053,0.052,0.024,-0.024,0.175,0.051,-0.073,-0.045,-0.079,-0.108,0.031,-0.070,-0.027,-0.017,0.131,0.010,-0.069,0.045,0.010,-0.041,-0.037,0.009,-0.006,-0.148,-0.038,-0.020,-0.062,-0.016,0.031,-0.035,-0.141,0.006,0.050,-0.058,-0.023,-0.069,-0.033,-0.133,0.006,0.045,-0.232,-0.135,-0.174,-0.120,-0.067,-0.078,-0.004,0.088,-0.102,0.065,-0.052,0.046,-0.013,0.125,0.044,0.037,0.036,0.033,0.101,0.062,0.131,0.081,0.045,-0.121,-0.045,0.104,-0.101,0.136,-0.048,-0.179,0.001,0.082,0.088,0.159,-0.094,0.127,0.078,-0.065,-0.017,0.045,0.123,0.017,-0.024,-0.004,0.155,-0.006,-0.165,0.049,-0.007,0.056,0.081,0.131,noHate
2,-0.026,-0.087,-0.084,0.070,-0.017,0.101,-0.173,0.040,0.049,0.054,0.025,-0.016,0.152,0.061,-0.084,-0.051,-0.040,-0.130,0.041,-0.041,-0.048,-0.030,0.139,-0.009,-0.071,0.050,0.014,-0.021,-0.047,0.023,-0.024,-0.165,-0.051,-0.005,-0.075,-0.006,0.075,-0.048,-0.161,-0.008,0.082,-0.046,-0.028,-0.079,-0.011,-0.136,0.030,0.035,-0.209,-0.124,-0.193,-0.156,-0.066,-0.077,-0.021,0.101,-0.108,0.069,-0.032,0.020,0.007,0.134,0.055,0.048,0.059,0.015,0.068,0.101,0.112,0.097,0.021,-0.128,-0.030,0.083,-0.115,0.146,-0.055,-0.164,0.008,0.064,0.077,0.138,-0.075,0.145,0.086,-0.060,-0.012,0.035,0.110,0.008,-0.015,0.006,0.168,-0.035,-0.140,0.049,0.024,0.086,0.099,0.137,noHate
3,-0.024,-0.062,-0.096,0.061,-0.041,0.088,-0.148,0.034,0.045,0.033,0.028,-0.039,0.165,0.066,-0.088,-0.049,-0.069,-0.126,0.021,-0.043,-0.050,-0.031,0.145,-0.014,-0.078,0.066,0.020,-0.029,-0.054,0.025,-0.009,-0.163,-0.037,0.001,-0.066,-0.020,0.067,-0.045,-0.151,-0.010,0.076,-0.057,-0.032,-0.056,-0.020,-0.139,0.011,0.041,-0.226,-0.138,-0.207,-0.156,-0.051,-0.073,-0.007,0.115,-0.104,0.076,-0.029,0.058,0.002,0.140,0.045,0.038,0.050,0.010,0.047,0.080,0.115,0.094,0.044,-0.104,-0.031,0.094,-0.112,0.129,-0.049,-0.175,-0.009,0.083,0.082,0.139,-0.104,0.115,0.062,-0.059,-0.020,0.053,0.093,-0.006,-0.032,0.002,0.166,-0.010,-0.135,0.040,0.016,0.083,0.105,0.141,noHate
4,-0.047,-0.079,-0.059,0.031,-0.026,0.139,-0.181,0.027,0.058,0.053,0.035,-0.021,0.188,0.052,-0.085,-0.031,-0.079,-0.118,0.029,-0.053,-0.035,-0.028,0.150,0.016,-0.050,0.045,-0.006,-0.046,-0.025,0.012,-0.006,-0.176,-0.046,-0.014,-0.073,-0.006,0.023,-0.040,-0.158,0.005,0.047,-0.069,0.001,-0.066,-0.036,-0.134,-0.004,0.024,-0.203,-0.119,-0.148,-0.099,-0.073,-0.082,0.008,0.072,-0.103,0.057,-0.064,0.048,0.010,0.113,0.039,0.038,0.031,0.038,0.088,0.071,0.125,0.061,0.058,-0.133,-0.067,0.112,-0.101,0.138,-0.058,-0.199,0.009,0.073,0.083,0.177,-0.085,0.123,0.072,-0.083,-0.045,0.035,0.129,0.030,-0.018,0.002,0.152,-0.011,-0.174,0.061,-0.003,0.051,0.082,0.137,noHate


In [71]:
x=df2.iloc[:,:-1]
y=df2.iloc[:,-1]

In [72]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=.2,random_state=1)

## Random Forest

In [73]:
# Used GridSearchCv And Trained With the best parameters 
rtree_eb1=RandomForestClassifier(n_estimators=100,max_depth=5,random_state=145,min_samples_split=5,max_leaf_nodes=30,min_samples_leaf=10,max_features=80)
rtree_eb1.fit(train_x,train_y)

RandomForestClassifier(max_depth=5, max_features=80, max_leaf_nodes=30,
                       min_samples_leaf=10, min_samples_split=5,
                       random_state=145)

In [74]:
print(rtree_eb1.score(train_x,train_y))
print(rtree_eb1.score(test_x,test_y))

0.8889142075880141
0.896127562642369


In [75]:
pred_rtree = rtree_eb1.predict(test_x)
pd.DataFrame(confusion_matrix(pred_rtree,test_y))

,0,1
0,0,0
1,228,1967


In [76]:
print(classification_report(test_y,pred_rtree))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.95      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM

In [77]:
# Used GridSearchCv And Trained With the best parameters 
svm_eb1=SVC(C=100,gamma=1,kernel='rbf')
svm_eb1.fit(train_x,train_y)

SVC(C=100, gamma=1)

In [78]:
print(svm_eb1.score(train_x,train_y))
print(svm_eb1.score(test_x,test_y))

0.8891420758801413
0.8956719817767653


In [79]:
pred_svm=svm_eb1.predict(test_x)
pd.DataFrame(confusion_matrix(pred_svm,test_y))

,0,1
0,0,1
1,228,1966


In [80]:
print(classification_report(test_y,pred_svm))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



## KNN

In [81]:
# Used GridSearchCv And Trained With the best parameters 
knn_eb1=KNeighborsClassifier(n_neighbors=25,weights='uniform',leaf_size=30)
knn_eb1.fit(train_x,train_y)

KNeighborsClassifier(n_neighbors=25)

In [82]:
print(knn_eb1.score(train_x,train_y))
print(knn_eb1.score(test_x,test_y))

0.8888002734419506
0.896127562642369


In [83]:
pred_knn=knn_eb1.predict(test_x)
pd.DataFrame(confusion_matrix(pred_knn,test_y))

,0,1
0,0,0
1,228,1967


In [84]:
print(classification_report(test_y,pred_knn))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.95      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## XG Boost

In [85]:
# Used GridSearchCv And Trained With the best parameters 
xgb_eb1=XGBClassifier(booster='gbtree',max_depth=4,n_estimator=50,random_state=67,learning_rate=0.01,min_child_weight=1.5,max_leaf_nodes=15,alpha=2)
xgb_eb1.fit(train_x,train_y)

[15:55:31] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { max_leaf_nodes, n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=2, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              max_leaf_nodes=15, min_child_weight=1.5, missing=nan,
              monotone_constraints='()', n_estimator=50, n_estimators=100,
              n_jobs=0, num_parallel_tree=1, random_state=67, reg_alpha=2,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [86]:
print(xgb_eb1.score(train_x,train_y))
print(xgb_eb1.score(test_x,test_y))

0.8889142075880141
0.896127562642369


In [87]:
pred_xgb=xgb_eb1.predict(test_x)
pd.DataFrame(confusion_matrix(pred_xgb,test_y))

,0,1
0,0,0
1,228,1967


In [88]:
print(classification_report(test_y,pred_xgb))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.95      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



## ADA Boost

In [89]:
clf_ada2 = AdaBoostClassifier(n_estimators=300,learning_rate=0.3,random_state=0,max_depth=5)
clf_ada2.fit(train_x,train_y)

print(clf_ada2.score(train_x,train_y))
print(clf_ada2.score(test_x,test_y))

pred_ada2 = clf_ada2.predict(test_x)
pd.DataFrame(confusion_matrix(pred_ada2,test_y))

0.8895978124643956
0.8956719817767653


,0,1
0,0,1
1,228,1966


In [90]:
print(classification_report(test_y,pred_ada2))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



## Gradient Boost

In [91]:

clf1_gb2 = GradientBoostingClassifier(random_state=0,n_estimators=300,learning_rate=0.01,max_depth=15)
clf1_gb2.fit(train_x,train_y)

print(clf1_gb2.score(train_x,train_y))
print(clf1_gb2.score(test_x,test_y))


pred_gb2 = clf1_gb2.predict(test_x)
pd.DataFrame(confusion_matrix(pred_gb2,test_y))

0.9998860658539365
0.8956719817767653


,0,1
0,1,2
1,227,1965


In [92]:
print(classification_report(test_y,pred_gb2))

              precision    recall  f1-score   support

        hate       0.33      0.00      0.01       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.90      2195
   macro avg       0.61      0.50      0.48      2195
weighted avg       0.84      0.90      0.85      2195



In [93]:
cm_random_forest_eb=confusion_matrix(test_y,pred_rtree)
cm_knn_eb=confusion_matrix(test_y,pred_knn)
cm_svm_eb=confusion_matrix(test_y,pred_svm)
cm_xgboost_eb =confusion_matrix(test_y,pred_xgb)
cm_adaboost_eb=confusion_matrix(pred_ada2,test_y)
cm_gradientboost_eb=confusion_matrix(pred_gb2,test_y)

In [94]:
print('Without Sampling')
print('------------------------')
print('Random Forest')
print('Training Accuracy :',rtree_eb1.score(train_x,train_y))
print('Testing Accuracy :',rtree_eb1.score(test_x,test_y))
print('-------------------------')
print('KNN')
print('Training Accuracy :',knn_eb1.score(train_x,train_y))
print('Testing Accuracy :',knn_eb1.score(test_x,test_y))
print('-------------------------')
print('SVM')

print('Training Accuracy :',svm_eb1.score(train_x,train_y))
print('Testing Accuracy :',svm_eb1.score(test_x,test_y))

print('-------------------------')
print('XGBoost')
print('Training Accuracy :',xgb_eb1.score(train_x,train_y))
print('Testing Accuracy :',xgb_eb1.score(test_x,test_y))

print('-------------------------')
print('AdaBoost')
print('Training Accuracy :',clf_ada2.score(train_x,train_y))
print('Testing Accuracy :',clf_ada2.score(test_x,test_y))

print('-------------------------')
print('GradientBoost')
print('Training Accuracy :',clf1_gb2.score(train_x,train_y))
print('Testing Accuracy :',clf1_gb2.score(test_x,test_y))

Without Sampling
------------------------
Random Forest
Training Accuracy : 0.8889142075880141
Testing Accuracy : 0.896127562642369
-------------------------
KNN
Training Accuracy : 0.8888002734419506
Testing Accuracy : 0.896127562642369
-------------------------
SVM
Training Accuracy : 0.8891420758801413
Testing Accuracy : 0.8956719817767653
-------------------------
XGBoost
Training Accuracy : 0.8889142075880141
Testing Accuracy : 0.896127562642369
-------------------------
AdaBoost
Training Accuracy : 0.8895978124643956
Testing Accuracy : 0.8956719817767653
-------------------------
GradientBoost
Training Accuracy : 0.9998860658539365
Testing Accuracy : 0.8956719817767653


In [95]:
print('Without Sampling')
print('-------------------------')
print('Confusion Maxtrix For Random Forest')
print(cm_random_forest_eb)
print('-----------------------------------')
print('Confusion Maxtrix For KNN')
print(cm_knn_eb)
print('--------------------------------------')
print('Confusion Maxtrix For SVM')
print(cm_svm_eb)
print('----------------------------')
print('Confusion Maxtrix For XGB')
print(cm_xgboost_eb)
print('----------------------------')
print('Confusion Maxtrix For AdaBoost')
print(cm_adaboost_eb)
print('----------------------------')
print('Confusion Maxtrix For GB')
print(cm_gradientboost_eb)

Without Sampling
-------------------------
Confusion Maxtrix For Random Forest
[[   0  228]
 [   0 1967]]
-----------------------------------
Confusion Maxtrix For KNN
[[   0  228]
 [   0 1967]]
--------------------------------------
Confusion Maxtrix For SVM
[[   0  228]
 [   1 1966]]
----------------------------
Confusion Maxtrix For XGB
[[   0  228]
 [   0 1967]]
----------------------------
Confusion Maxtrix For AdaBoost
[[   0    1]
 [ 228 1966]]
----------------------------
Confusion Maxtrix For GB
[[   1    2]
 [ 227 1965]]


# Using Word2Vec with Sampling

In [96]:
x_eb=df2.iloc[:,:-1]
y_eb=df2.iloc[:,-1]

In [97]:
x_ros,y_ros=ros.fit_sample(x_eb,y_eb)
x_train_eb,x_test_eb,y_train_eb,y_test_eb = train_test_split(x_ros,y_ros,test_size=.2,random_state=1)

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


## Random Forest

In [98]:
# Used GridSearchCv And Trained With the best parameters 
rtree_eb3=RandomForestClassifier(max_depth=11,n_estimators=300,min_samples_split=5,max_leaf_nodes=30,min_samples_leaf=10,max_features=80)
rtree_eb3.fit(x_train_eb,y_train_eb)

RandomForestClassifier(max_depth=11, max_features=80, max_leaf_nodes=30,
                       min_samples_leaf=10, min_samples_split=5,
                       n_estimators=300)

In [99]:
print(rtree_eb3.score(x_train_eb,y_train_eb))
print(rtree_eb3.score(x_test_eb,y_test_eb))

0.7294305822136916
0.7205731832139202


In [100]:
pred_rtree_eb = rtree_eb3.predict(x_test_eb)
pd.DataFrame(confusion_matrix(pred_rtree_eb,y_test_eb))

,0,1
0,1642,806
1,286,1174


In [101]:
print(classification_report(y_test_eb,pred_rtree_eb))

              precision    recall  f1-score   support

        hate       0.67      0.85      0.75      1928
      noHate       0.80      0.59      0.68      1980

    accuracy                           0.72      3908
   macro avg       0.74      0.72      0.72      3908
weighted avg       0.74      0.72      0.72      3908



## SVM

In [102]:
# Used GridSearchCv And Trained With the best parameters 
svm_eb3=SVC(C=100,gamma=10,kernel='rbf')
svm_eb3.fit(x_train_eb,y_train_eb)

SVC(C=100, gamma=10)

In [103]:
print(svm_eb3.score(x_train_eb,y_train_eb))
print(svm_eb3.score(x_test_eb,y_test_eb))

0.8364043506078055
0.808085977482088


In [104]:
pred_svm_eb = svm_eb3.predict(x_test_eb)
pd.DataFrame(confusion_matrix(pred_svm_eb,y_test_eb))

,0,1
0,1839,661
1,89,1319


In [105]:
print(classification_report(y_test_eb,pred_svm_eb))

              precision    recall  f1-score   support

        hate       0.74      0.95      0.83      1928
      noHate       0.94      0.67      0.78      1980

    accuracy                           0.81      3908
   macro avg       0.84      0.81      0.80      3908
weighted avg       0.84      0.81      0.80      3908



## KNN

In [106]:
# Used GridSearchCv And Trained With the best parameters 
knn_eb3=KNeighborsClassifier(n_neighbors=5)
knn_eb3.fit(x_train_eb,y_train_eb)

KNeighborsClassifier()

In [107]:
print(knn_eb3.score(x_train_eb,y_train_eb))
print(knn_eb3.score(x_test_eb,y_test_eb))

0.8603326935380678
0.8024564994882293


In [108]:
pred_knn_eb = knn_eb3.predict(x_test_eb)
pd.DataFrame(confusion_matrix(pred_knn_eb,y_test_eb))

,0,1
0,1890,734
1,38,1246


In [109]:
print(classification_report(y_test_eb,pred_knn_eb))

              precision    recall  f1-score   support

        hate       0.72      0.98      0.83      1928
      noHate       0.97      0.63      0.76      1980

    accuracy                           0.80      3908
   macro avg       0.85      0.80      0.80      3908
weighted avg       0.85      0.80      0.80      3908



## XG Boost

In [110]:
# Used GridSearchCv And Trained With the best parameters 
xgb_eb3=XGBClassifier(booster='gbtree',max_depth=17,n_estimator=450,random_state=12345,learning_rate=0.01)
xgb_eb3.fit(x_train_eb,y_train_eb)

[16:01:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=17,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimator=450, n_estimators=100, n_jobs=0, num_parallel_tree=1,
              random_state=12345, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [111]:
print(xgb_eb3.score(x_train_eb,y_train_eb))
print(xgb_eb3.score(x_test_eb,y_test_eb))

0.9177223288547665
0.8866427840327533


In [112]:
pred_xgb_eb = xgb_eb3.predict(x_test_eb)
pd.DataFrame(confusion_matrix(pred_xgb_eb,y_test_eb))

,0,1
0,1927,442
1,1,1538


In [113]:
print(classification_report(y_test_eb,pred_xgb_eb))

              precision    recall  f1-score   support

        hate       0.81      1.00      0.90      1928
      noHate       1.00      0.78      0.87      1980

    accuracy                           0.89      3908
   macro avg       0.91      0.89      0.89      3908
weighted avg       0.91      0.89      0.89      3908



## ADA Boost

In [114]:
clf_ada3 = AdaBoostClassifier(n_estimators=200,learning_rate=0.9,random_state=0)
clf_ada3.fit(x_train_eb,y_train_eb)

print(clf_ada3.score(x_train_eb,y_train_eb))
print(clf_ada3.score(x_test_eb,y_test_eb))

pred_ada3 = clf_ada3.predict(x_test_eb)
pd.DataFrame(confusion_matrix(pred_ada3,y_test_eb))

0.7312220089571337
0.7195496417604913


,0,1
0,1568,736
1,360,1244


In [115]:
print(classification_report(y_test_eb,pred_ada3))

              precision    recall  f1-score   support

        hate       0.68      0.81      0.74      1928
      noHate       0.78      0.63      0.69      1980

    accuracy                           0.72      3908
   macro avg       0.73      0.72      0.72      3908
weighted avg       0.73      0.72      0.72      3908



## Gradient Boost

In [116]:
from sklearn.ensemble import GradientBoostingClassifier
clf1_gb3 = GradientBoostingClassifier(random_state=10,n_estimators=250,learning_rate=0.1,max_depth=9)
clf1_gb3.fit(x_train_eb,y_train_eb)

print(clf1_gb3.score(x_train_eb,y_train_eb))
print(clf1_gb3.score(x_test_eb,y_test_eb))


pred_gb3 = clf1_gb3.predict(x_test_eb)
pd.DataFrame(confusion_matrix(pred_gb3,y_test_eb))

0.9992962252079335
0.9966734902763562


,0,1
0,1927,12
1,1,1968


In [117]:
print(classification_report(y_test_eb,pred_gb3))

              precision    recall  f1-score   support

        hate       0.99      1.00      1.00      1928
      noHate       1.00      0.99      1.00      1980

    accuracy                           1.00      3908
   macro avg       1.00      1.00      1.00      3908
weighted avg       1.00      1.00      1.00      3908



In [118]:
cm_random_forest_eb_sm=confusion_matrix(y_test_eb,pred_rtree_eb)
cm_knn_eb_sm=confusion_matrix(y_test_eb,pred_knn_eb)
cm_svm_eb_sm=confusion_matrix(y_test_eb,pred_svm_eb)
cm_xgboost_eb_sm =confusion_matrix(y_test_eb,pred_xgb_eb)
cm_adaboost_eb_sm=confusion_matrix(pred_ada3,y_test_eb)
cm_gradientboost_eb_sm=confusion_matrix(pred_gb3,y_test_eb)

In [119]:
print('With Sampling')
print('------------------------')
print('Random Forest')
print('Training Accuracy :',rtree_eb3.score(x_train_eb,y_train_eb))
print('Testing Accuracy :',rtree_eb3.score(x_test_eb,y_test_eb))
print('-------------------------')
print('KNN')
print('Training Accuracy :',knn_eb3.score(x_train_eb,y_train_eb))
print('Testing Accuracy :',knn_eb3.score(x_test_eb,y_test_eb))
print('-------------------------')
print('SVM')

print('Training Accuracy :',svm_eb3.score(x_train_eb,y_train_eb))
print('Testing Accuracy :',svm_eb3.score(x_test_eb,y_test_eb))

print('-------------------------')
print('XGBoost')
print('Training Accuracy :',xgb_eb3.score(x_train_eb,y_train_eb))
print('Testing Accuracy :',xgb_eb3.score(x_test_eb,y_test_eb))

print('-------------------------')
print('AdaBoost')
print('Training Accuracy :',clf_ada3.score(x_train_eb,y_train_eb))
print('Testing Accuracy :',clf_ada3.score(x_test_eb,y_test_eb))

print('-------------------------')
print('GradientBoost')
print('Training Accuracy :',clf1_gb3.score(x_train_eb,y_train_eb))
print('Testing Accuracy :',clf1_gb3.score(x_test_eb,y_test_eb))

With Sampling
------------------------
Random Forest
Training Accuracy : 0.7294305822136916
Testing Accuracy : 0.7205731832139202
-------------------------
KNN
Training Accuracy : 0.8603326935380678
Testing Accuracy : 0.8024564994882293
-------------------------
SVM
Training Accuracy : 0.8364043506078055
Testing Accuracy : 0.808085977482088
-------------------------
XGBoost
Training Accuracy : 0.9177223288547665
Testing Accuracy : 0.8866427840327533
-------------------------
AdaBoost
Training Accuracy : 0.7312220089571337
Testing Accuracy : 0.7195496417604913
-------------------------
GradientBoost
Training Accuracy : 0.9992962252079335
Testing Accuracy : 0.9966734902763562


In [120]:
print('With Sampling')
print('-------------------------')
print('Confusion Maxtrix For Random Forest')
print(cm_random_forest_eb_sm)
print('-----------------------------------')
print('Confusion Maxtrix For KNN')
print(cm_knn_eb_sm)
print('--------------------------------------')
print('Confusion Maxtrix For SVM')
print(cm_svm_eb_sm)
print('----------------------------')
print('Confusion Maxtrix For XGB')
print(cm_xgboost_eb_sm)
print('----------------------------')
print('Confusion Maxtrix For AdaBoost')
print(cm_adaboost_eb_sm)
print('----------------------------')
print('Confusion Maxtrix For GB')
print(cm_gradientboost_eb_sm)

With Sampling
-------------------------
Confusion Maxtrix For Random Forest
[[1642  286]
 [ 806 1174]]
-----------------------------------
Confusion Maxtrix For KNN
[[1890   38]
 [ 734 1246]]
--------------------------------------
Confusion Maxtrix For SVM
[[1839   89]
 [ 661 1319]]
----------------------------
Confusion Maxtrix For XGB
[[1927    1]
 [ 442 1538]]
----------------------------
Confusion Maxtrix For AdaBoost
[[1568  736]
 [ 360 1244]]
----------------------------
Confusion Maxtrix For GB
[[1927   12]
 [   1 1968]]


# FASTTEXT

In [121]:
from gensim.models.fasttext import FastText

In [122]:
embedding_size = 100
window_size = 40
min_word = 5
down_sampling = 1e-2

In [123]:
ft_model = FastText(list_of_sent,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=1,
                      iter=100)

In [124]:
def make_feature_vec(words):
  #for sent in list_of_sent:
  feature_vec=np.zeros((100,),dtype='float32')
  nwords = 0
  index2word_set= set(ft_model.wv.index2word) 
  for word in words:
    if word in index2word_set:
        nwords +=1
        feature_vec=np.add(feature_vec,ft_model[word])     
  feature_vec=np.divide(feature_vec,nwords)
  feature_vec = np.around(feature_vec,3)
  return feature_vec

In [125]:
def get_avg_feature_vec(tweets):
    c=0
    tweet_feature_vec = np.zeros((len(tweets),100),dtype='float32')
    for tweet in tweets:
      tweet_feature_vec[c]=make_feature_vec(tweet)
      c=c+1
    return tweet_feature_vec
#tweets=[]
#for tweet in df['tweet']:
  #tweets.append(tweet)
x= get_avg_feature_vec(train_data['tweet'])

C:\Users\pradeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
C:\Users\pradeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [126]:
data=pd.DataFrame(x)
data['labels']=train_data['label']

In [127]:
data.dropna(inplace=True)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,labels
0,0.044,-0.027,-0.006,-0.071,-0.052,0.028,0.012,0.005,-0.016,0.033,0.006,-0.014,0.017,-0.027,0.008,0.015,0.007,0.010,0.065,-0.040,-0.028,0.008,-0.005,0.023,0.005,0.004,0.022,-0.014,-0.025,0.033,-0.036,-0.001,0.002,-0.036,0.012,0.016,0.023,0.005,0.009,0.009,0.002,0.010,-0.005,0.024,-0.002,-0.005,-0.006,0.008,0.028,-0.031,-0.021,-0.000,-0.014,-0.002,0.018,-0.020,-0.001,0.004,-0.017,0.025,-0.029,0.003,0.008,0.002,0.011,0.006,-0.003,-0.012,-0.004,0.002,-0.012,-0.011,-0.000,0.011,-0.002,-0.008,-0.010,0.031,0.007,-0.038,0.007,-0.016,0.027,-0.033,0.014,-0.017,0.001,-0.024,0.002,-0.025,-0.009,-0.036,0.003,0.004,-0.006,0.023,0.053,-0.073,-0.028,-0.004,noHate
1,0.047,-0.028,-0.004,-0.062,-0.048,0.029,0.010,-0.000,-0.011,0.033,0.014,-0.019,0.019,-0.027,0.006,0.009,0.006,0.009,0.064,-0.040,-0.028,0.009,-0.006,0.022,0.009,0.005,0.019,-0.013,-0.025,0.029,-0.034,0.003,0.001,-0.034,0.011,0.014,0.024,0.006,0.009,0.014,-0.001,0.011,-0.008,0.017,-0.006,-0.007,-0.009,0.009,0.031,-0.027,-0.020,-0.001,-0.016,-0.001,0.017,-0.018,-0.001,-0.000,-0.015,0.029,-0.026,-0.003,0.009,0.004,0.011,0.003,-0.000,-0.006,0.003,-0.002,-0.006,-0.006,0.001,0.015,-0.002,-0.008,-0.013,0.027,0.011,-0.036,0.004,-0.014,0.029,-0.031,0.022,-0.019,-0.004,-0.024,0.003,-0.032,-0.013,-0.042,0.009,0.001,-0.010,0.025,0.046,-0.072,-0.023,-0.005,noHate
2,0.046,-0.024,-0.004,-0.063,-0.051,0.030,0.014,0.001,-0.013,0.034,0.013,-0.019,0.023,-0.029,0.005,0.013,0.011,0.010,0.063,-0.038,-0.028,0.008,-0.006,0.022,0.008,0.004,0.020,-0.012,-0.025,0.029,-0.037,0.001,0.001,-0.034,0.011,0.015,0.026,0.006,0.008,0.013,-0.000,0.013,-0.007,0.022,-0.006,-0.008,-0.009,0.010,0.029,-0.029,-0.021,-0.001,-0.016,-0.001,0.018,-0.019,-0.002,0.002,-0.014,0.026,-0.024,-0.001,0.008,0.004,0.007,0.007,0.001,-0.005,-0.003,-0.001,-0.009,-0.007,-0.000,0.014,-0.003,-0.007,-0.006,0.026,0.007,-0.031,0.004,-0.018,0.033,-0.034,0.021,-0.016,0.001,-0.030,0.009,-0.029,-0.011,-0.045,0.005,0.005,-0.004,0.022,0.045,-0.067,-0.022,-0.007,noHate
3,0.048,-0.020,0.003,-0.066,-0.052,0.033,0.013,-0.004,-0.012,0.034,0.014,-0.021,0.020,-0.027,0.001,0.010,0.011,0.012,0.063,-0.039,-0.029,0.011,-0.005,0.015,0.009,0.005,0.019,-0.010,-0.024,0.026,-0.034,0.004,0.002,-0.036,0.011,0.017,0.026,0.007,0.006,0.010,-0.001,0.013,-0.008,0.019,-0.006,-0.007,-0.009,0.009,0.028,-0.028,-0.020,-0.001,-0.015,-0.001,0.017,-0.018,-0.001,-0.000,-0.011,0.026,-0.021,-0.001,0.008,0.006,0.009,0.005,0.002,-0.004,-0.000,-0.003,-0.006,-0.006,0.001,0.018,-0.004,-0.008,-0.008,0.023,0.008,-0.028,0.003,-0.013,0.031,-0.032,0.028,-0.015,0.000,-0.030,0.005,-0.032,-0.014,-0.050,0.010,-0.002,-0.011,0.024,0.041,-0.068,-0.023,-0.011,noHate
4,0.043,-0.027,-0.003,-0.070,-0.056,0.029,0.012,0.003,-0.012,0.033,0.014,-0.017,0.019,-0.021,0.005,0.007,0.012,0.014,0.062,-0.040,-0.027,0.009,-0.004,0.026,0.007,0.000,0.019,-0.015,-0.029,0.033,-0.041,-0.001,0.002,-0.035,0.011,0.016,0.022,0.007,0.008,0.009,0.001,0.011,-0.007,0.022,-0.006,-0.007,-0.010,0.010,0.031,-0.029,-0.021,-0.001,-0.016,-0.001,0.018,-0.019,-0.002,0.005,-0.017,0.026,-0.028,0.001,0.008,0.002,0.009,0.005,-0.001,-0.008,-0.000,0.000,-0.008,-0.007,-0.001,0.012,-0.003,-0.007,-0.010,0.032,0.007,-0.040,0.006,-0.019,0.030,-0.034,0.015,-0.017,-0.001,-0.025,0.003,-0.026,-0.009,-0.035,0.006,0.005,-0.008,0.019,0.048,-0.065,-0.021,-0.003,noHate


In [128]:
x1=data.iloc[:,:-1]
y1=data.iloc[:,-1]
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=.2,random_state=1)

## RandomForest

In [129]:
rtree_ft=RandomForestClassifier(n_estimators=250,max_depth=5,random_state=12,min_samples_split=5,max_leaf_nodes=40,min_samples_leaf=10,max_features=80)
rtree_ft.fit(x_train1,y_train1)

RandomForestClassifier(max_depth=5, max_features=80, max_leaf_nodes=40,
                       min_samples_leaf=10, min_samples_split=5,
                       n_estimators=250, random_state=12)

In [130]:
print('Random Forest')
print('Training Accuracy :',rtree_ft.score(x_train1,y_train1))
print('Testing Accuracy :',rtree_ft.score(x_test1,y_test1))

Random Forest
Training Accuracy : 0.8889142075880141
Testing Accuracy : 0.896127562642369


In [131]:
pred_rtree_ft = rtree_ft.predict(x_test1)
pd.DataFrame(confusion_matrix(y_test1,pred_rtree_ft))

,0,1
0,0,228
1,0,1967


In [132]:
print(classification_report(y_test1,pred_rtree_ft))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.95      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## XGBoost

In [133]:
xgb_ft=XGBClassifier(booster='gbtree',max_depth=4,n_estimators=300,learning_rate=0.05)
xgb_ft.fit(x_train1,y_train1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [134]:
print('XGBoost')
print('Training Accuracy :',xgb_ft.score(x_train1,y_train1))
print('Testing Accuracy :',xgb_ft.score(x_test1,y_test1))

XGBoost
Training Accuracy : 0.8922182978238579
Testing Accuracy : 0.8956719817767653


In [135]:
pred_xgb_ft=xgb_ft.predict(x_test1)
pd.DataFrame(confusion_matrix(y_test1,pred_xgb_ft))

,0,1
0,0,228
1,1,1966


In [136]:
print(classification_report(y_test1,pred_xgb_ft))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



## SVM

In [137]:
svm_ft=SVC(C=300,gamma=20,kernel='rbf')
svm_ft.fit(x_train1,y_train1)

SVC(C=300, gamma=20)

In [138]:
print('SVM')

print('Training Accuracy :',svm_ft.score(x_train1,y_train1))
print('Testing Accuracy :',svm_ft.score(x_test1,y_test1))

SVM
Training Accuracy : 0.8893699441722684
Testing Accuracy : 0.8956719817767653


In [139]:
pred_svm_ft=svm_ft.predict(x_test1)
pd.DataFrame(confusion_matrix(y_test1,pred_svm_ft))

,0,1
0,0,228
1,1,1966


In [140]:
print(classification_report(y_test1,pred_svm_ft))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



# KNN

In [141]:
# Used GridSearchCv And Trained With the best parameters 
knn_ft=KNeighborsClassifier(n_neighbors=15)
knn_ft.fit(x_train1,y_train1)

KNeighborsClassifier(n_neighbors=15)

In [142]:
print('KNN')
print('Training Accuracy :',knn_ft.score(x_train1,y_train1))
print('Testing Accuracy :',knn_ft.score(x_test1,y_test1))

KNN
Training Accuracy : 0.8905092856329042
Testing Accuracy : 0.8943052391799544


In [143]:
pred_knn_ft=knn_ft.predict(x_test1)
pd.DataFrame(confusion_matrix(y_test1,pred_knn_ft))

,0,1
0,3,225
1,7,1960


In [144]:
print(classification_report(y_test1,pred_knn_ft))

              precision    recall  f1-score   support

        hate       0.30      0.01      0.03       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.89      2195
   macro avg       0.60      0.50      0.48      2195
weighted avg       0.84      0.89      0.85      2195



## ADABoost

In [145]:
from sklearn.ensemble import AdaBoostClassifier
clf_ft = AdaBoostClassifier(n_estimators=350,learning_rate=0.6,random_state=0)
clf_ft.fit(x_train1,y_train1)

print(clf_ft.score(x_train1,y_train1))
print(clf_ft.score(x_test1,y_test1))

adapred_ft=clf_ft.predict(x_test1)
pd.DataFrame(confusion_matrix(adapred_ft,y_test1))

0.8919904295317307
0.8933940774487471


,0,1
0,2,8
1,226,1959


In [146]:
print(classification_report(y_test1,adapred_ft))

              precision    recall  f1-score   support

        hate       0.20      0.01      0.02       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.89      2195
   macro avg       0.55      0.50      0.48      2195
weighted avg       0.82      0.89      0.85      2195



## GradientBoost

In [147]:
from sklearn.ensemble import GradientBoostingClassifier
clf1_ft = GradientBoostingClassifier(random_state=0,n_estimators=200,learning_rate=0.01,max_depth=4)
clf1_ft.fit(x_train1,y_train1)

print(clf1_ft.score(x_train1,y_train1))
print(clf1_ft.score(x_test1,y_test1))

gbpred_ft=clf1_ft.predict(x_test1)
pd.DataFrame(confusion_matrix(gbpred_ft,y_test1))

0.8892560100262048
0.8956719817767653


,0,1
0,0,1
1,228,1966


In [148]:
print(classification_report(y_test1,gbpred_ft))

              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       228
      noHate       0.90      1.00      0.94      1967

    accuracy                           0.90      2195
   macro avg       0.45      0.50      0.47      2195
weighted avg       0.80      0.90      0.85      2195



In [149]:
cm_random_forest_ft=confusion_matrix(y_test1,pred_rtree_ft)
cm_knn_ft=confusion_matrix(y_test1,pred_knn_ft)
cm_svm_ft=confusion_matrix(y_test1,pred_svm_ft)
cm_xgboost_ft =confusion_matrix(y_test1,pred_xgb_ft)
cm_adaboost_ft=confusion_matrix(adapred_ft,y_test1)
cm_gradientboost_ft=confusion_matrix(gbpred_ft,y_test1)

In [150]:
print('Without Sampling')
print('------------------------')
print('Random Forest')
print('Training Accuracy :',rtree_ft.score(x_train1,y_train1))
print('Testing Accuracy :',rtree_ft.score(x_test1,y_test1))
print('-------------------------')
print('KNN')
print('Training Accuracy :',knn_ft.score(x_train1,y_train1))
print('Testing Accuracy :',knn_ft.score(x_test1,y_test1))
print('-------------------------')
print('SVM')

print('Training Accuracy :',svm_ft.score(x_train1,y_train1))
print('Testing Accuracy :',svm_ft.score(x_test1,y_test1))

print('-------------------------')
print('XGBoost')
print('Training Accuracy :',xgb_ft.score(x_train1,y_train1))
print('Testing Accuracy :',xgb_ft.score(x_test1,y_test1))

print('-------------------------')
print('AdaBoost')
print('Training Accuracy :',clf_ft.score(x_train1,y_train1))
print('Testing Accuracy :',clf_ft.score(x_test1,y_test1))

print('-------------------------')
print('GradientBoost')
print('Training Accuracy :',clf1_ft.score(x_train1,y_train1))
print('Testing Accuracy :',clf1_ft.score(x_test1,y_test1))

Without Sampling
------------------------
Random Forest
Training Accuracy : 0.8889142075880141
Testing Accuracy : 0.896127562642369
-------------------------
KNN
Training Accuracy : 0.8905092856329042
Testing Accuracy : 0.8943052391799544
-------------------------
SVM
Training Accuracy : 0.8893699441722684
Testing Accuracy : 0.8956719817767653
-------------------------
XGBoost
Training Accuracy : 0.8922182978238579
Testing Accuracy : 0.8956719817767653
-------------------------
AdaBoost
Training Accuracy : 0.8919904295317307
Testing Accuracy : 0.8933940774487471
-------------------------
GradientBoost
Training Accuracy : 0.8892560100262048
Testing Accuracy : 0.8956719817767653


In [151]:
print('Without Sampling')
print('-------------------------')
print('Confusion Maxtrix For Random Forest')
print(cm_random_forest_ft)
print('-----------------------------------')
print('Confusion Maxtrix For KNN')
print(cm_knn_ft)
print('--------------------------------------')
print('Confusion Maxtrix For SVM')
print(cm_svm_ft)
print('----------------------------')
print('Confusion Maxtrix For XGB')
print(cm_xgboost_ft)
print('----------------------------')
print('Confusion Maxtrix For AdaBoost')
print(cm_adaboost_ft)
print('----------------------------')
print('Confusion Maxtrix For GB')
print(cm_gradientboost_ft)

Without Sampling
-------------------------
Confusion Maxtrix For Random Forest
[[   0  228]
 [   0 1967]]
-----------------------------------
Confusion Maxtrix For KNN
[[   3  225]
 [   7 1960]]
--------------------------------------
Confusion Maxtrix For SVM
[[   0  228]
 [   1 1966]]
----------------------------
Confusion Maxtrix For XGB
[[   0  228]
 [   1 1966]]
----------------------------
Confusion Maxtrix For AdaBoost
[[   2    8]
 [ 226 1959]]
----------------------------
Confusion Maxtrix For GB
[[   0    1]
 [ 228 1966]]


## FastText with Sampling


In [152]:
x_ft=data.iloc[:,:-1]
y_ft=data.iloc[:,-1]

In [153]:
x_ros1,y_ros1=ros.fit_sample(x_ft,y_ft)
x_train_ft,x_test_ft,y_train_ft,y_test_ft = train_test_split(x_ros1,y_ros1,test_size=.2,random_state=1)

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


## Random Forest

In [154]:

# Used GridSearchCv And Trained With the best parameters 
rtree_ft1=RandomForestClassifier(max_depth=9,n_estimators=300,random_state=8)
rtree_ft1.fit(x_train_ft,y_train_ft)

RandomForestClassifier(max_depth=9, n_estimators=300, random_state=8)

In [155]:
print('Random Forest')
print('Training Accuracy :',rtree_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',rtree_ft1.score(x_test_ft,y_test_ft))

Random Forest
Training Accuracy : 0.7971849008317339
Testing Accuracy : 0.7945240532241555


In [156]:
pred_rtree_ft1 = rtree_ft1.predict(x_test_ft)
pd.DataFrame(confusion_matrix(y_test_ft,pred_rtree_ft1))

,0,1
0,1925,3
1,800,1180


In [157]:
print(classification_report(y_test_ft,pred_rtree_ft1))

              precision    recall  f1-score   support

        hate       0.71      1.00      0.83      1928
      noHate       1.00      0.60      0.75      1980

    accuracy                           0.79      3908
   macro avg       0.85      0.80      0.79      3908
weighted avg       0.85      0.79      0.79      3908



## XGBoost 

In [158]:
xgb_ft1=XGBClassifier(booster='gbtree',max_depth=8,n_estimators=300,learning_rate=0.01)
xgb_ft1.fit(x_train_ft,y_train_ft)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [159]:
print('XGBoost')
print('Training Accuracy :',xgb_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',xgb_ft1.score(x_test_ft,y_test_ft))

XGBoost
Training Accuracy : 0.85489443378119
Testing Accuracy : 0.8457011258955988


In [160]:
pred_xgb_ft1=xgb_ft1.predict(x_test_ft)
pd.DataFrame(confusion_matrix(y_test_ft,pred_xgb_ft1))

,0,1
0,1901,27
1,576,1404


In [161]:
print(classification_report(y_test_ft,pred_xgb_ft1))

              precision    recall  f1-score   support

        hate       0.77      0.99      0.86      1928
      noHate       0.98      0.71      0.82      1980

    accuracy                           0.85      3908
   macro avg       0.87      0.85      0.84      3908
weighted avg       0.88      0.85      0.84      3908



## SVM

In [162]:
svm_ft1=SVC(C=100,gamma=5,kernel='rbf')
svm_ft1.fit(x_train_ft,y_train_ft)

SVC(C=100, gamma=5)

In [163]:
print('SVM')

print('Training Accuracy :',svm_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',svm_ft1.score(x_test_ft,y_test_ft))

SVM
Training Accuracy : 0.6570057581573896
Testing Accuracy : 0.6550665301944729


In [164]:
pred_svm_ft1=svm_ft1.predict(x_test_ft)
pd.DataFrame(confusion_matrix(y_test_ft,pred_svm_ft1))

,0,1
0,1656,272
1,1076,904


In [165]:
print(classification_report(y_test_ft,pred_svm_ft1))

              precision    recall  f1-score   support

        hate       0.61      0.86      0.71      1928
      noHate       0.77      0.46      0.57      1980

    accuracy                           0.66      3908
   macro avg       0.69      0.66      0.64      3908
weighted avg       0.69      0.66      0.64      3908



## KNN 

In [166]:
# Used GridSearchCv And Trained With the best parameters 
knn_ft1=KNeighborsClassifier(n_neighbors=10)
knn_ft1.fit(x_train_ft,y_train_ft)

KNeighborsClassifier(n_neighbors=10)

In [167]:
print('KNN')
print('Training Accuracy :',knn_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',knn_ft1.score(x_test_ft,y_test_ft))

KNN
Training Accuracy : 0.7414587332053743
Testing Accuracy : 0.6914022517911975


In [168]:
pred_knn_ft1=knn_ft.predict(x_test_ft)
pd.DataFrame(confusion_matrix(y_test_ft,pred_knn_ft1))

,0,1
0,46,1882
1,4,1976


In [169]:
print(classification_report(y_test_ft,pred_knn_ft1))

              precision    recall  f1-score   support

        hate       0.92      0.02      0.05      1928
      noHate       0.51      1.00      0.68      1980

    accuracy                           0.52      3908
   macro avg       0.72      0.51      0.36      3908
weighted avg       0.71      0.52      0.37      3908



## AdaBoost

In [170]:
from sklearn.ensemble import AdaBoostClassifier
clf_ft1 = AdaBoostClassifier(n_estimators=300,learning_rate=0.5,random_state=0)
clf_ft1.fit(x_train_ft,y_train_ft)

print(clf_ft1.score(x_train_ft,y_train_ft))
print(clf_ft1.score(x_test_ft,y_test_ft))

adapred_ft1=clf_ft1.predict(x_test_ft)
pd.DataFrame(confusion_matrix(adapred_ft1,y_test_ft))

0.7257197696737044
0.7123848515864892


,0,1
0,1555,751
1,373,1229


In [171]:
print(classification_report(y_test_ft,adapred_ft1))

              precision    recall  f1-score   support

        hate       0.67      0.81      0.73      1928
      noHate       0.77      0.62      0.69      1980

    accuracy                           0.71      3908
   macro avg       0.72      0.71      0.71      3908
weighted avg       0.72      0.71      0.71      3908



## Gradient Boost

In [172]:
from sklearn.ensemble import GradientBoostingClassifier
clf1_ft1 = GradientBoostingClassifier(random_state=10,n_estimators=300,learning_rate=0.1,max_depth=9)
clf1_ft1.fit(x_train_ft,y_train_ft)

print(clf1_ft1.score(x_train_ft,y_train_ft))
print(clf1_ft1.score(x_test_ft,y_test_ft))

gbpred_ft1=clf1_ft1.predict(x_test_ft)
pd.DataFrame(confusion_matrix(gbpred_ft1,y_test_ft))

0.9992962252079335
0.996417604912999


,0,1
0,1927,13
1,1,1967


In [173]:
print(classification_report(y_test_ft,gbpred_ft1))

              precision    recall  f1-score   support

        hate       0.99      1.00      1.00      1928
      noHate       1.00      0.99      1.00      1980

    accuracy                           1.00      3908
   macro avg       1.00      1.00      1.00      3908
weighted avg       1.00      1.00      1.00      3908



In [174]:
cm_random_forest_ft1=confusion_matrix(y_test_ft,pred_rtree_ft1)
cm_knn_ft1=confusion_matrix(y_test_ft,pred_knn_ft1)
cm_svm_ft1=confusion_matrix(y_test_ft,pred_svm_ft1)
cm_xgboost_ft1 =confusion_matrix(y_test_ft,pred_xgb_ft1)
cm_adaboost_ft1=confusion_matrix(adapred_ft1,y_test_ft)
cm_gradientboost_ft1=confusion_matrix(gbpred_ft1,y_test_ft)

In [175]:
print('Without Sampling')
print('------------------------')
print('Random Forest')
print('Training Accuracy :',rtree_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',rtree_ft1.score(x_test_ft,y_test_ft))
print('-------------------------')
print('KNN')
print('Training Accuracy :',knn_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',knn_ft1.score(x_test_ft,y_test_ft))
print('-------------------------')
print('SVM')

print('Training Accuracy :',svm_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',svm_ft1.score(x_test_ft,y_test_ft))

print('-------------------------')
print('XGBoost')
print('Training Accuracy :',xgb_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',xgb_ft1.score(x_test_ft,y_test_ft))

print('-------------------------')
print('AdaBoost')
print('Training Accuracy :',clf_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',clf_ft1.score(x_test_ft,y_test_ft))

print('-------------------------')
print('GradientBoost')
print('Training Accuracy :',clf1_ft1.score(x_train_ft,y_train_ft))
print('Testing Accuracy :',clf1_ft1.score(x_test_ft,y_test_ft))

Without Sampling
------------------------
Random Forest
Training Accuracy : 0.7971849008317339
Testing Accuracy : 0.7945240532241555
-------------------------
KNN
Training Accuracy : 0.7414587332053743
Testing Accuracy : 0.6914022517911975
-------------------------
SVM
Training Accuracy : 0.6570057581573896
Testing Accuracy : 0.6550665301944729
-------------------------
XGBoost
Training Accuracy : 0.85489443378119
Testing Accuracy : 0.8457011258955988
-------------------------
AdaBoost
Training Accuracy : 0.7257197696737044
Testing Accuracy : 0.7123848515864892
-------------------------
GradientBoost
Training Accuracy : 0.9992962252079335
Testing Accuracy : 0.996417604912999


In [176]:
print('With Sampling')
print('-------------------------')
print('Confusion Maxtrix For Random Forest')
print(cm_random_forest_ft1)
print('-----------------------------------')
print('Confusion Maxtrix For KNN')
print(cm_knn_ft1)
print('--------------------------------------')
print('Confusion Maxtrix For SVM')
print(cm_svm_ft1)
print('----------------------------')
print('Confusion Maxtrix For XGB')
print(cm_xgboost_ft1)
print('----------------------------')
print('Confusion Maxtrix For AdaBoost')
print(cm_adaboost_ft1)
print('----------------------------')
print('Confusion Maxtrix For GB')
print(cm_gradientboost_ft1)

With Sampling
-------------------------
Confusion Maxtrix For Random Forest
[[1925    3]
 [ 800 1180]]
-----------------------------------
Confusion Maxtrix For KNN
[[  46 1882]
 [   4 1976]]
--------------------------------------
Confusion Maxtrix For SVM
[[1656  272]
 [1076  904]]
----------------------------
Confusion Maxtrix For XGB
[[1901   27]
 [ 576 1404]]
----------------------------
Confusion Maxtrix For AdaBoost
[[1555  751]
 [ 373 1229]]
----------------------------
Confusion Maxtrix For GB
[[1927   13]
 [   1 1967]]


In [177]:
import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(clf_ada, 'tfidf.pkl') 

['tfidf.pkl']

In [235]:
# Save the model as a pickle in a file 
import joblib
joblib.dump(svm_tf2, 'tfidfsamp10000.pkl')  

['tfidfsamp10000.pkl']

In [179]:
# Save the model as a pickle in a file 
joblib.dump(clf1_gb2, 'word2vec.pkl') 

['word2vec.pkl']

In [180]:
# Save the model as a pickle in a file 
joblib.dump(clf1_gb3, 'word2vecsamp.pkl') 

['word2vecsamp.pkl']

In [181]:
# Save the model as a pickle in a file 
joblib.dump(xgb_ft, 'fasttext.pkl') 

['fasttext.pkl']

In [182]:
# Save the model as a pickle in a file 
joblib.dump(clf1_ft1, 'fasttextsamp.pkl') 

['fasttextsamp.pkl']